In [58]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from bs4 import BeautifulSoup
import unicodedata
import contractions
import google_list
import re
import nltk
import spacy
import math
from string import ascii_uppercase
from string import ascii_lowercase
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models
import copy

In [59]:
lda = models.LdaModel.load('lrnr/best_lda_topic_model.gensim')

In [60]:
corpus = pd.read_csv('l_corpus.csv')

In [61]:
lrnr = pd.read_csv('lrnr/learners.csv')

In [62]:
lrnr['contribution_id'] =  lrnr['contribution_id'].map(lambda x: str(x))

In [63]:
corp = corpus.drop(['Unnamed: 0','description','clean_text','tokens'],axis=1)

In [64]:
corp = corp.rename(columns={'resource_id':'contribution_id'})

In [65]:
corp['contribution_id'] = corp['contribution_id'].map(lambda x: x[7:len(x)-4])

In [66]:
result = pd.merge(lrnr, corp, how='inner', on=['contribution_id','contribution_id'])

In [67]:
result

,contribution_id,user_id,mandate,type,order,date_time,full_text
0,1,m68,mandate_1,Classroom log,1,"3 September 2018, 9:46 PM",Network Science for the Web Classroom Log - 3...
1,2,m21,mandate_1,Classroom log,1,"21 August 2018, 1:57 PM",A Presentation By Anish Pal (MT2017021) Networ...
2,3,37,mandate_1,essay,1,"18 September 2018, 12:48 PM",R.Lakshmi Priya Prof. Srinath Srinivasa 18 Se...
3,4,m63,mandate_1,essay,1,"18 September 2018, 11:52 AM","In Statistics, Exploratory Data Analysis(EDA) ..."
4,5,m35,mandate_1,essay,1,"18 September 2018, 4:28 AM",what is Manifold Learning ..? Manifold learni...
5,6,m5,mandate_1,essay,1,"18 September 2018, 12:48 PM","Data Analytics In layman terms, analytics ref..."
6,7,m55,mandate_1,essay,1,"25 August 2018, 6:54 AM",what is Manifold Learning ..? First of all wha...
7,8,prof,mandate_1,reply/feedback,2,"10 September 2018, 4:39 PM",Technical content of the essay can be greatly ...
8,9,14,mandate_1,discussion,1,"10 September 2018, 3:17 PM",Whatsapp has more number of users than telegra...
9,10,prof,mandate_1,discussion,2,"10 September 2018, 4:35 PM",Question needs more details. What are the more...


In [68]:
# result = result[result['user_id'] != 'prof']

In [69]:
result

,contribution_id,user_id,mandate,type,order,date_time,full_text
0,1,m68,mandate_1,Classroom log,1,"3 September 2018, 9:46 PM",Network Science for the Web Classroom Log - 3...
1,2,m21,mandate_1,Classroom log,1,"21 August 2018, 1:57 PM",A Presentation By Anish Pal (MT2017021) Networ...
2,3,37,mandate_1,essay,1,"18 September 2018, 12:48 PM",R.Lakshmi Priya Prof. Srinath Srinivasa 18 Se...
3,4,m63,mandate_1,essay,1,"18 September 2018, 11:52 AM","In Statistics, Exploratory Data Analysis(EDA) ..."
4,5,m35,mandate_1,essay,1,"18 September 2018, 4:28 AM",what is Manifold Learning ..? Manifold learni...
5,6,m5,mandate_1,essay,1,"18 September 2018, 12:48 PM","Data Analytics In layman terms, analytics ref..."
6,7,m55,mandate_1,essay,1,"25 August 2018, 6:54 AM",what is Manifold Learning ..? First of all wha...
7,8,prof,mandate_1,reply/feedback,2,"10 September 2018, 4:39 PM",Technical content of the essay can be greatly ...
8,9,14,mandate_1,discussion,1,"10 September 2018, 3:17 PM",Whatsapp has more number of users than telegra...
9,10,prof,mandate_1,discussion,2,"10 September 2018, 4:35 PM",Question needs more details. What are the more...


In [70]:
result.to_csv('learners_contribution.csv')